<a href="https://colab.research.google.com/github/HomeschoolHarry/forecast_demand_planning/blob/main/Seasonal_Combined_ARIMA_forecasts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import os
!pip install darts

import numpy as np
import requests
import seaborn as sns
import pandas as pd
import torch
import matplotlib.pyplot as plt

from darts import TimeSeries
from darts.utils.timeseries_generation import (
    gaussian_timeseries,
    linear_timeseries,
    sine_timeseries,
)
from darts.models import (
    RNNModel,
    TCNModel,
    TransformerModel,
    NBEATSModel,
    BlockRNNModel,
    VARIMA,
    NaiveDrift,
    NaiveSeasonal
)
from darts.metrics import mape, smape, mae
from darts.dataprocessing.transformers import Scaler
from darts.utils.timeseries_generation import datetime_attribute_timeseries

import logging

logging.disable(logging.CRITICAL)

import warnings

warnings.filterwarnings("ignore")

%matplotlib inline

# for reproducibility
torch.manual_seed(1)
np.random.seed(1)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


API

In [4]:
# Grab Tito's 750mL using item Code and All-Other A/O category sales
# use "where funtion to fix 2012 through Dec 2022"
url = f"https://data.iowa.gov/resource/m3tr-qhgy.json?$select=date_trunc_ym(date) as date,itemno='38176' as tito_750,sum(sale_liters) as volume&$group=date,tito_750&$where=date between '2012-01-01T00:00:00' and '2022-12-31T00:00:00'"
response = requests.get(url)
response.raise_for_status()

# To save as Pandas DataFrame
titos = response.json()
titos = pd.DataFrame(titos)

#convert types
convert_dict = {'date': object,
                'volume': float,
                }
titos = titos.astype(convert_dict)

# use numpy to convert year and month to datetime
titos['date'] = pd.to_datetime(titos['date'])

# grab 750 ml using True value
tito_750_only = titos[titos['tito_750'] == True]
#drop the boolean column
tito_750_only = tito_750_only.drop('tito_750', axis=1)

#grab all-other sales
ao_sales = titos[titos['tito_750'] == False]

# Volume DataFrames
df_tito_volume = tito_750_only[['date', 'volume']]

# convert to standard 9L cases
def conv_nine(row):
  return (row['volume'] / 9).round(decimals=0)

df_tito_volume['volume (9L)'] = conv_nine(df_tito_volume)

df_tito_volume = df_tito_volume.drop(columns='volume')

### Create series and Train set

In [5]:
series_tito_volume = TimeSeries.from_dataframe(df_tito_volume, time_col='date')

train = series_tito_volume

### Create Weighted Values

In [19]:
df_tito_volume.

,date,volume (9L)
1,2012-01-01,58.0
3,2012-02-01,117.0
5,2012-03-01,93.0
7,2012-04-01,108.0
9,2012-05-01,195.0
...,...,...
255,2022-08-01,2221.0
257,2022-09-01,3371.0
259,2022-10-01,2883.0
261,2022-11-01,2479.0


### SEASONAL NAIVE

In [17]:
seasonal_model = NaiveSeasonal(K=12)
seasonal_model.fit(train)
seasonal_forecast_12 = seasonal_model.predict(12) #12 Months

# convert to dataframe
seasonal_forecast_12.pd_dataframe()

#convert to CSV

component,volume (9L)
date,
2023-01-01,2076.0
2023-02-01,1343.0
2023-03-01,2872.0
2023-04-01,2034.0
2023-05-01,2324.0
2023-06-01,4352.0
2023-07-01,2511.0
2023-08-01,2221.0
2023-09-01,3371.0


## COMBINED

In [ ]:
drift_model = NaiveDrift()
drift_model.fit(train)
drift_forecast = drift_model.predict(12) #12 months

combined_forecast = drift_forecast + seasonal_forecast_12 - train.last_value()

val.plot(label="actual")
combined_forecast.plot(label="combined")
drift_forecast.plot(label="drift")

print(
    "Mean absolute percentage error for the combined naive drift + seasonal: {:.2f}%.".format(
        mape(val, combined_forecast)
    )
)

\